In [1]:
from __future__ import absolute_import 
from __future__ import division
from __future__ import print_function

import os 
import tempfile
from import_ipynb import *

import Tokenization 
import six 
import tensorflow as tf 



importing Jupyter notebook from Tokenization.ipynb


In [ ]:
# Xây dựng lớp tokenization test 
class TokenizationTest(tf.test.TestCase):

    # xaay duwngj phuwowng thuwcs thwur nghieemj ddaayf đủ cho tất cả các tokens . 
    def test_full_tokenizer(self):
        # xây dựng một từ điển các tokens là danh sách các tokens cần đặc biệt 
        vocab_tokens = [
              "[UNK]", "[CLS]", "[SEP]", "want", "##want", "##ed", "wa", "un", "runn",
                "##ing", ","
        ]
        # sử dụng hàm tempfile để tạo 1 tệp tạm thời có tên và sử dụng cấu trúc with để đảm 
        # bảo các tệp được đóng lịa khi không cần thiết 
        # với delete = False nghĩa là tệp tạm thời không bị xóa đi khi đóng 
        # mà phải được xóa thủ công bằng os.unlink , vocan_write là một đối tượng têoj , có thể 
        # được sử dụng để ghi nội dung vào tệp tạm thời 
        with tempfile.NamedTemporaryFile(delete=False) as vocab_writer: 
            # kiểm tra phiên bản python hiện tại . 
            if six.PY2: 
                # neeus nos alf py2 vaf ghi nội dung của tập vocab_tokens vào tập tam thời 
                # vocab_wite mỗi phần tử được thêm vò một ký tự xuống dòng 
                vocab_writer.write("".join([x + "\n" for x in vocab_tokens]))
            else :  # Trường hợp với python 3 ta cũng làm tương tự 
                vocab_writer.write("".join(
            [x + "\n" for x in vocab_tokens]).encode("utf-8"))

        # sau đó gán tên của tệp tạm thời cho biến vocab_file 
        vocab_file = vocab_writer.name
    
        # Truyền file nguồn vào tokenizer  
        tokenizer = Tokenization.FullTokenizer(vocab_file)
        # xóa tệp vocab_file 
        os.unlink(vocab_file)
        
        # Xử lý file nguồn với token hóa với các định dnagj tiêu chuẩn 
        # với u là chuỗi vaen bản viết theo dnagj unicode 
        # \u00E9 là một ký tự đặc biệt, được mã hóa theo định dạng Unicode, tương ứng với ký tự d
        # UNwanted và running, mà BERT sẽ tách thành các token nhỏ hơn như sau:
        #  ["un", "##want", "##ed", ",", "runn", "##ing"].
        tokens = Tokenization.tokenize(u"UNwant\u00E9d,running")
        # sau đó sử dụng phương thức assertAllEqual của lớp tf.test.testcase để kiểm tra xem 
        # kết quả của tokenize có đúng với dnah sách các tokens được mong đợi không. 
        self.assertAllEqual(tokens , ["un", "##want", "##ed", ",", "runn", "##ing"])
        # chuyển đổi các tokens tương ứng với định dạng thành các id tươnh ứng trong tập 
        # từ vựng và kiểm tra xem kết quả có đúng với danh sách id mong muốn .
        self.assertAllEqual(
            tokenizer.convert_tokens_to_ids(tokens), [7, 4, 5, 10, 8, 9])


    # xây dựng phương thức thử nghiệm các dạng ký tự tiếng trung 
    def test_chinese(self):
        # Tạo  biến tokenizer với nhiệm vụ thực hiện xử lý các định dạng căn bản 
        tokenizer = Tokenization.BasicTokenizer()

        # sử dụng phương thức assertAllEqual để so sánh kết u=quả của phương thức tokenize với 
        #  danh sách kỳ vọng Ví dụ với chuỗi u"ah\u535A\u63A8zz"  sẽ được token hóa thành 
        # [u"ah", u"\u535A", u"\u63A8", u"zz"] với u"ah\u535A\u63A8zz" là  2 ký tự tiếng hán
        self.assertAllEqual(
            tokenizer.tokenize(u"ah\u535A\u63A8zz"),
                [u"ah", u"\u535A", u"\u63A8", u"zz"]
        )

    # xây dựng phương thức thử nghiệm token hóa chữ thường cơ bản 
    def test_basic_tokenizer_lower(self):
        # xuwr lys văn bản với chữ thường các kỹ tự chuyển về thường trước khi token hóa 
        tokenizer = Tokenization.BassicTokenizer(do_lower_case=True)

        # sử dụng phương thức assertAllEqual để so sánh kết quả của phương thức tokenize
        # với một danh sách kỳ vọng : Ví dụ chuỗi u" \tHeLLo!how  \n Are yoU?  " sẽ được token 
        # hóa thành ["hello", "!", "how", "are", "you", "?"]
        self.assertAllEqual(
            tokenizer.tokenize(u" \tHeLLo!how  \n Are yoU?  "),
                ["hello", "!", "how", "are", "you", "?"])
        self.assertAllEqual(tokenizer.tokenize(u"H\u00E9llo"), ["hello"])

    # xây dựng phươnh thức thử nghiệm token hóa với định dạng nguyên bản 
    def test_basic_tokenizer_no_lower(self):
        # thực hiện xây dụng tuowng tự như tokenizer lowe
        tokenizer = Tokenization.BasicTokenizer(do_lower_case=False)

        # Hàm này cũng sử dụng phương thức assertAllEqual để so sánh kết quả của phương thức
        #  tokenize với một danh sách kỳ vọng. Ví dụ, chuỗi u" \tHeLLo!how  \n Are yoU?  
        # " sẽ được token hóa thành ["HeLLo", "!", "how", "Are", "yoU", "?"],
        #  trong đó các ký tự hoa đã được giữ nguyên và tất cả các khoảng trắng và ký tự xuống dòng đã được loại bỏ.
        self.assertAllEqual(
            tokenizer.tokenize(u" \tHeLLo!how  \n Are yoU?  "),
                ["HeLLo", "!", "how", "Are", "yoU", "?"])
        
    # Xây dượng phương thức thử nghiệm mã hóa token vơí WordPiece 
    def test_wordpiece_tokenizer(self):
        # tạo một bộ danh sách từ vựng 
        vocab_tokens = [
            "[UNK]", "[CLS]", "[SEP]" , "want" ,"##want" ,"##ed" , "wa" , 
            "un" , "runn" , "##ing"
        ]
        # tạo một từ điển rỗng 
        vocab = {}
        # duyệt qua một danh sách các chỉ só  và giá trị của danh sách vocab_tokens đã thiết lập 
        for (i, token) in enumerate (vocab_tokens):
            # đặt các chỉn số i với giá trị tương ứng trong danh sách vocab tokens 
            # đưa tất cả vào từ điển vocab 
            vocab[token] = i
            # sử dụng phương thức assertAllEqual để so sánh kết của phương thức convert_token_to_ids 
            # với danh sách kỳ vọng  ["un", "##want", "##ed", "runn", "##ing"]), [7, 4, 5, 8, 9] 
            self.assertAllEqual(
                Tokenization.convert_tokens_to_ids(
                    vocab ,  ["un", "##want", "##ed", "runn", "##ing"]), [7, 4, 5, 8, 9]
                )
    
    # Xây dựng các phuuwong thức xử lý thử nghiệm với ký tự điều khiển , các định dạng ký tự
    # đặc biệt cũng như với các khoảng trắng 
    def test_is_whitespace(self):
        self.assertTrue(Tokenization._is_whitespace(u" "))
        self.assertTrue(Tokenization._is_whitespace(u"\t"))
        self.assertTrue(Tokenization._is_whitespace(u"\r"))
        self.assertTrue(Tokenization._is_whitespace(u"\n"))
        self.assertTrue(Tokenization._is_whitespace(u"\u00A0"))

        self.assertFalse(Tokenization._is_whitespace(u"A"))
        self.assertFalse(Tokenization._is_whitespace(u"-"))

    def test_is_control(self):
        self.assertTrue(Tokenization._is_control(u"\u0005"))

        self.assertFalse(Tokenization._is_control(u"A"))
        self.assertFalse(Tokenization._is_control(u" "))
        self.assertFalse(Tokenization._is_control(u"\t"))
        self.assertFalse(Tokenization._is_control(u"\r"))
        self.assertFalse(Tokenization._is_control(u"\U0001F4A9"))

    def test_is_punctuation(self):
        self.assertTrue(Tokenization._is_punctuation(u"-"))
        self.assertTrue(Tokenization._is_punctuation(u"$"))
        self.assertTrue(Tokenization._is_punctuation(u"`"))
        self.assertTrue(Tokenization._is_punctuation(u"."))

        self.assertFalse(Tokenization._is_punctuation(u"A"))
        self.assertFalse(Tokenization._is_punctuation(u" "))


if __name__ == "__main__":
  tf.test.main()